In [1]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *
from procurement_lib import redash, send_slack_notification, GoogleSheet

import random
import time
import datetime

In [2]:
df_grads = get_fresh_query_result("https://internal-redash.federate.frubana.com/",199393,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_fmg_comex = get_fresh_query_result("https://internal-redash.federate.frubana.com/",199394,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_kam = get_fresh_query_result("https://internal-redash.federate.frubana.com/",199396,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_fmg_br = get_fresh_query_result("https://internal-redash.federate.frubana.com/",199392,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200
200
200
200


In [3]:
def gmv(num):    
    query = """
    SELECT 
        s.identifier_value AS city,
        TO_CHAR(fs.order_close_date, 'YYYY')::int AS year,
        TO_CHAR(fs.order_close_date, 'MM')::int AS month,
        TO_CHAR(fs.order_close_date, 'WW')::int AS weekk,
        CASE
            WHEN fsd.adjustment_reason ILIKE '%kam%' THEN 'KAM'
            WHEN fsd.adjustment_reason ILIKE 'hoo%' THEN 'HOOK'
            WHEN fsd.adjustment_reason ILIKE '%\_NB\_%' THEN 'SUPER DISCOUNTS'
            WHEN fsd.adjustment_reason ILIKE 'sh\_%' THEN 'SUPER HOOKS'
            WHEN fsd.adjustment_reason ILIKE 'EXTRA\_INV%' THEN 'ADDITIONAL INVESTMENT'
            WHEN (fsd.adjustment_reason ILIKE '%merma%' OR fsd.adjustment_reason ILIKE '%acmk%' OR fsd.adjustment_reason ILIKE '%liquidation%' OR fsd.adjustment_reason IS NULL) THEN 'NO GROWTH'
        ELSE 'OTHERS' END AS responsable,
        dc.source_id AS customer_id,
        SUM(fs.gmv) AS gmv_usd,
        SUM((CASE WHEN (fsd.adjustment_reason NOT ILIKE '%merma%' AND fsd.adjustment_reason NOT ILIKE '%acmk%' AND fsd.adjustment_reason NOT ILIKE '%liquidation%') THEN fsd.product_discount ELSE 0 END)*cr.coefficient) AS discount_applied
    FROM dpr_sales.fact_sales                       fs
    INNER JOIN dpr_shared.dim_customer              dc  ON dc.customer_id = fs.dim_customer
    INNER JOIN dpr_shared.dim_site                  s   ON s.site_id = fs.dim_site
    INNER JOIN dpr_shared.dim_product               dp  ON dp.product_id = fs.dim_product
    INNER JOIN dpr_shared.dim_currency_conversion   cr  ON cr.currency_id = fs.dim_currency AND cr.start_date_id = 20180101
    LEFT JOIN dpr_sales.fact_sales_discounts        fsd ON fs.order_item_id = fsd.order_item_id 

    WHERE 
        fs.gmv_enabled = TRUE
        AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
        AND fs.fb_order_status_id  IN (1,6,7,8)
        AND fs.is_deleted = FALSE
        AND fs.dim_status = 1
        AND dp.is_slot = 'false'
        AND fs.order_close_date >= DATE_TRUNC('month', date(getdate()) - interval '{number} month')
        AND fs.order_close_date < DATE_TRUNC('month', date(getdate()) - interval '{number2} month')
    GROUP BY 1,2,3,4,5,6
    """.format(number=num, number2=num-1)
    df_gmv = run_read_dwd_query(query)
    df_gmv[["gmv_usd","discount_applied"]] = df_gmv[["gmv_usd","discount_applied"]].astype(float)

    return df_gmv

In [4]:
# Create an empty list to store DataFrames
dfs = []

# Iterate through the range and append each result to the list
for i in range(0, 1):
    df_gmv = gmv(i)
    dfs.append(df_gmv)
    print(i)
    
# Concatenate the list of DataFrames into a single DataFrame
result_df = pd.concat(dfs, ignore_index=True)

0


In [5]:
result_df.month.unique()

array([3])

In [6]:
df2 =  pd.merge(result_df, df_kam, left_on=['city','weekk','customer_id'], right_on=['region_code','week','customer_id'], how='left')
df2 =  pd.merge(df2, df_fmg_comex, left_on=['city','weekk','customer_id'], right_on=['region_code','week_list','customer_id'], how='left')
df2 =  pd.merge(df2, df_fmg_br, left_on=['weekk','customer_id'], right_on=['week','customer_id'], how='left')

In [7]:
df_aux = df2[['customer_id','weekk','year_x']].drop_duplicates()

In [8]:
# Initialize 'is_graduated' column with 0
df_aux['is_graduated'] = 0

# Iterate over each row in df_aux
for index, row in df_aux.iterrows():
    i+=1
    customer_id = row['customer_id']
    week = row['weekk']
    year = row['year_x']

    # Check if customer_id is in df_grads
    if customer_id in df_grads['uid'].values:
        # Get relevant information from df_grads for the current customer
        customer_grads = df_grads[df_grads['uid'] == customer_id].iloc[0]

        # Experimental condition
        exp_condition = (
            (week >= customer_grads['week_grad']) & (year == customer_grads['year_grad']) or
            (year > customer_grads['year_grad'])
        )

        # Desgrad condition
        desgrad_condition = (
            (week >= customer_grads['week_grad']) & (year == customer_grads['year_grad']) or
            (year > customer_grads['year_grad'])
        ) and (
            (week <= customer_grads['week_desgrad']) & (year == customer_grads['year_desgrad']) or
            (year < customer_grads['year_desgrad'])
        )

        # Update 'is_graduated' column for the current row and week
        df_aux.loc[index, 'is_graduated'] = int(exp_condition or desgrad_condition)
        print(i)

3
4
6
10
12
15
16
19
20
22
24
25
26
27
28
29
30
31
32
33
34
35
37
38
40
41
42
43
44
46
47
49
51
56
57
59
60
62
64
65
66
67
70
71
72
73
76
77
78
79
82
83
84
85
87
89
91
93
95
96
97
101
102
103
105
107
108
110
111
113
114
115
117
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
141
142
143
144
145
146
147
149
151
152
153
154
156
157
159
160
161
162
163
164
167
169
170
171
172
173
176
177
178
179
180
181
182
183
184
186
188
191
192
193
194
195
198
200
202
204
206
207
208
209
211
212
213
215
216
217
218
223
224
225
226
227
228
229
230
232
233
234
236
237
238
239
240
242
243
244
245
246
247
249
250
251
252
254
257
258
259
260
261
263
264
265
266
268
269
270
272
273
274
277
280
281
282
283
284
285
286
288
289
290
291
292
293
294
295
296
298
300
302
306
308
311
312
313
315
316
318
319
320
321
323
324
326
327
328
329
330
331
334
335
336
338
339
340
343
346
347
350
351
352
355
356
357
358
359
360
361
364
366
367
368
371
372
373
375
376
378
379
381
382
383
385
386
387


In [9]:
df2 =  pd.merge(df2, df_aux, left_on=['weekk','customer_id','year_x'], right_on=['weekk','customer_id','year_x'], how='left')

In [10]:
df3 = df2[['city', 'month', 'weekk', 'responsable', 'customer_id', 'gmv_usd', 
     'discount_applied', 'is_kam', 'is_farming_x', 'is_farming_y', 'is_graduated']].copy()

In [11]:
df3 = df3.fillna(0)

In [12]:
df3['check'] = df3.is_farming_x+df3.is_farming_y+df3.is_graduated+df3.is_kam

In [13]:
df3.check.value_counts()

1.0    108325
0.0      9772
2.0      5269
3.0         1
Name: check, dtype: int64

In [14]:
def determine_group(row):
    if row['check'] == 1:
        if row['is_farming_x'] == 1 or row['is_farming_y'] == 1:
            return 'farming'
        elif row['is_kam'] == 1:
            return 'kam'
        elif row['is_graduated'] == 1:
            return 'grad'
    elif row['check'] == 0:
        return 'other'
    elif row['check'] == 2:
        if row['is_kam'] == 1:
            return 'kam'
        elif row['is_graduated'] == 1:
            return 'grad'
        else:
            return 'farming'
    elif row['check'] == 3:
        if row['is_kam'] == 1:
            return 'kam'
        elif row['is_graduated'] == 1:
            return 'grad'
        else:
            return 'farming'
    else:
        return np.nan

# Apply the custom function to create the 'group' column
df3['group'] = df3.apply(determine_group, axis=1)

In [15]:
df3.month.unique()

array([3])

In [16]:
df4 = df3.groupby(['city','month','weekk','responsable','group']).agg({'gmv_usd':np.sum,'discount_applied':np.sum}).reset_index()

In [17]:
# Assuming your DataFrame is named 'df4'
df4['country'] = df4['city'].map({
    'BOG': 'CO',
    'BAQ': 'CO',
    'MDE': 'CO',
    'SPO': 'BR',
    'BHZ': 'BR',
    'CWB': 'BR',
    'VCP': 'BR',
    'CMX': 'MX'
})

# Create a new DataFrame for country-level aggregation
country_df4 = df4.groupby(['country', 'month','weekk', 'responsable', 'group']).agg({
    'gmv_usd': 'sum',
    'discount_applied': 'sum'
}).reset_index()
country_df4.rename(columns={'country': 'city'}, inplace=True)

# Create a DataFrame for LATAM-level aggregation
latam_df4 = df4.groupby(['month','weekk', 'responsable', 'group']).agg({
    'gmv_usd': 'sum',
    'discount_applied': 'sum'
}).reset_index()
latam_df4['city'] = 'LATAM'

df4.drop(['country'], axis=1, inplace=True)

In [18]:
# Append the aggregated country-level data to the original DataFrame
df4 = pd.concat([df4, country_df4], ignore_index=True, sort=False)

# Append the aggregated LATAM-level data to the original DataFrame
df4 = pd.concat([df4, latam_df4[['city','month','weekk', 'responsable', 'group', 'gmv_usd', 'discount_applied']]], ignore_index=True, sort=False)

# Sort the DataFrame based on the original index to preserve the order
df4.sort_index(inplace=True)

In [19]:
# Create a DataFrame for 'responsable' named 'ALL'
all_df = df4.groupby(['month','weekk', 'responsable', 'city']).agg({
    'gmv_usd': 'sum',
    'discount_applied': 'sum'
}).reset_index()
all_df['group'] = 'ALL'

# Append the aggregated 'ALL' data to the original DataFrame
df4 = pd.concat([df4, all_df[['city','month','weekk', 'responsable', 'group', 'gmv_usd', 'discount_applied']]], ignore_index=True, sort=False)

# Sort the DataFrame based on the original index to preserve the order
df4.sort_index(inplace=True)

In [20]:
# Create a DataFrame for 'responsable' named 'ALL'
all_df = df4.groupby(['month','weekk', 'group', 'city']).agg({
    'gmv_usd': 'sum',
    'discount_applied': 'sum'
}).reset_index()
all_df['responsable'] = 'ALL'

# Append the aggregated 'ALL' data to the original DataFrame
df4 = pd.concat([df4, all_df[['city','month','weekk', 'responsable', 'group', 'gmv_usd', 'discount_applied']]], ignore_index=True, sort=False)

# Sort the DataFrame based on the original index to preserve the order
df4.sort_index(inplace=True)

In [21]:
df4.month.unique()

array([3])

In [22]:
# Assuming your DataFrame is named 'df'
df4['total_gmv'] = df4.groupby(['city', 'month','weekk', 'group'])['gmv_usd'].transform('sum')
df4['total_gmv'] = df4['total_gmv']/2
df4['discount_percentage'] = (df4['discount_applied'] / df4['total_gmv']) * 100

In [23]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   city                 858 non-null    object 
 1   month                858 non-null    int64  
 2   weekk                858 non-null    int64  
 3   responsable          858 non-null    object 
 4   group                858 non-null    object 
 5   gmv_usd              858 non-null    float64
 6   discount_applied     858 non-null    float64
 7   total_gmv            858 non-null    float64
 8   discount_percentage  858 non-null    float64
dtypes: float64(4), int64(2), object(3)
memory usage: 60.5+ KB


In [24]:
paramss_csv=GoogleSheet("1pVqWRHSn3qFtyrCz65GWFHvI2aHT1zDgqz8gP0MbmBo")
city_paramss = paramss_csv.get_as_dataframe('INFO')

In [25]:
paramss_csv.set_with_dataframe(df4, worksheet='INFO', autocreate=True)

In [26]:
#df4.to_excel('aaaa.xlsx')